# LeNet-5 CNN with Keras:
** I am developing a series of kernels for different Deep Learning Models: **

* [L-Layered Neural Network from scratch](https://www.kaggle.com/curiousprogrammer/l-layered-neural-network-from-scratch)
* [TensorFlow NN with Augmentation](https://www.kaggle.com/curiousprogrammer/digit-recognizer-tensorflow-nn-with-augmentation)
* [Data Augmentation in Python, TF, Keras, Imgaug](https://www.kaggle.com/curiousprogrammer/data-augmentation-in-python-tf-keras-imgaug)
* [Deep NN with Keras](https://www.kaggle.com/curiousprogrammer/deep-nn-with-keras-97-5)
* [CNN with TensorFlow](https://www.kaggle.com/curiousprogrammer/lenet-5-cnn-with-tensorflow-98-5)
* CNN with Keras - This one
* AutoEncoders with TensorFlow
* AutoEncoders with Keras
* GANs with TensorFlow
* GANs with Keras

In [ ]:
if 0:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if 0:
    !cp -r /content/drive/MyDrive/HomeWork5 /content/

In [ ]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#kaggle datasets download -d athota1/caltech101
if 0:
  ! pip install --upgrade --force-reinstall --no-deps kaggle
  from google.colab import files
  files.upload()
  ! mkdir ~/.kaggle
  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json
  ! kaggle competitions download -c digit-recognizer

  ! unzip digit-recognizer.zip -d /content/digit-recognizer


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "/content/digit-recognizer/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


# Any results you write to the current directory are saved as output.

# Load Data

In [ ]:
X_train = np.load('/content/HomeWork5/data/DataForClassification_TimeDomain.npy')
X_train = np.transpose(X_train)
print(X_train.shape)

(936, 3600)


In [ ]:
#types=9={'healthy','missing','crack','spall','chip5a','chip4a','chip3a','chip2a','chip1a'}

In [ ]:
Y_train = np.zeros((936,1))
label=0
print(label)
for i in range(936):

  Y_train[i]=label
  if (i%104==0) and (i!=0):
     label = label +1
     print(label)

0
1
2
3
4
5
6
7
8


In [ ]:
Y_train[104:103*2]

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes = 9)

In [ ]:
print(Y_train[101,:])

[1. 0. 0. 0. 0. 0. 0. 0. 0.]


# Plot Digits

In [ ]:
#Train-Test Split
X_dev, X_test, Y_dev, Y_test = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, random_state=0)

X_train, X_val, Y_train, Y_val = train_test_split(X_dev, Y_dev, test_size=0.2, shuffle=True, random_state=0)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(598, 3600)
(150, 3600)
(188, 3600)


In [ ]:
X_train = np.expand_dims(X_train, axis=1)
X_val = np.expand_dims(X_val, axis=1)
X_test = np.expand_dims(X_test, axis=1)
Y_train = np.expand_dims(Y_train, axis=1)
Y_val = np.expand_dims(Y_val, axis=1)
Y_test = np.expand_dims(Y_test, axis=1)

In [ ]:
import tensorflow_addons as tfa
from keras import layers

In [ ]:
model = Sequential()

model.add(layers.GRU(32, input_shape=(1, 3600 ), return_sequences=True))

model.add(layers.Dense(8))
model.add(Dense(9, activation='softmax')) #burayi 9 yapacaksaniz

In [ ]:
model.build()
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_18 (GRU)                (None, 1, 64)             703872    
                                                                 
 dense_35 (Dense)            (None, 1, 32)             2080      
                                                                 
 dense_36 (Dense)            (None, 1, 9)              297       
                                                                 
Total params: 706,249
Trainable params: 706,249
Non-trainable params: 0
_________________________________________________________________


In [ ]:
adam = Adam(lr=1e-3)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Set a learning rate annealer
import tensorflow as tf


model_name_save= '/content/nn_model.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name_save, save_freq='epoch', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = tf.keras.callbacks.EarlyStopping(  monitor='val_accuracy', patience=12, verbose=1,  mode='max',restore_best_weights=False)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(  monitor='val_accuracy', factor=0.3, patience=7,  min_lr=1e-5, verbose=1,  mode='max')


In [ ]:
X_train.shape

(598, 1, 3600)

In [ ]:
model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=50, batch_size=64)

# Let's predict test data

In [ ]:
Y_pred = model.predict(X_test, batch_size=1)
Y_pred = Y_pred.squeeze()

188/188 [==============================] - 1s 2ms/step
